## **Jaccard similarity and LDA-Topic Modelling as a Feature Extraction Technique**

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords

# <font color = red> Importing the Data Set

In [ ]:
DATASET_PATH = os.path.join(os.getcwd(),"C:\\Users\\Shreyansh\\Abhinav\\data","")

In [ ]:
Train_data = pd.read_csv(r'C:\Users\Shreyansh\Capstone_files\real_final_test_data.csv' , encoding='ISO-8859-1')

In [ ]:
Train_data.columns

Index(['Unnamed: 0', 'title', 'text', 'summary', 'keywords', 'headline',
       'domain', 'Category'],
      dtype='object')

In [ ]:
Train_data['Category'].nunique()

4

In [ ]:
Train_data = Train_data.rename(columns = {'headline': 'Headline', 'text': 'articleBody'}, inplace = False)

In [ ]:
Train_data = Train_data.rename(columns = {'Category':'Stance'}, inplace = False)

In [ ]:
print('The number of rows ',Train_data.shape[0])
print('The number of columns',Train_data.shape[1])

The number of rows  212
The number of columns 8


In [ ]:
Train_data["Headline"].nunique()

25

In [ ]:
Train_data.columns

Index(['Unnamed: 0', 'title', 'articleBody', 'summary', 'keywords', 'Headline',
       'domain', 'Stance'],
      dtype='object')

In [ ]:
Train_data = Train_data[['Headline', 'Stance', 'articleBody']]

In [ ]:
Train_data.shape

(212, 3)

# <font color = red> Text Preprocessing

In [ ]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer# Text Cleaning And Preprocessing
def clean_text(df):
    all_reviews = list()
    
    # Taking all the text from the Data Frame
    lines = df.values.tolist()
    
    #Iterating Through All the Documents
    for text in lines:
        text = str(text).lower()          # Convert into Lower case
        
        
        #Removing all the URL Links from the Corpus and replace them with empty or common string
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        
        #Removing the Emojis and replacing them from empty or common string
        emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text = emoji.sub(r'', text)
        
        
        # Expanding Common Short Forms
        # Not very Helpful in Sentimental Analysis
        # These are pronoun and verbs.
        # Important if you want to create a text from your model along with Classifying the sentinment.
        # Helpful when you are creating Chatbot.
        
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"that's", "that is", text)        
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"where's", "where is", text) 
        text = re.sub(r"\'ll", " will", text)  
        text = re.sub(r"\'ve", " have", text)  
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"did't", "did not", text)
        text = re.sub(r"can't", "can not", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"have't", "have not", text)
        
        # Removing All the Punctuation
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        
        # Tokenize the Sentences(Split the sentences by Spaces)
        #(Can also Use Split Function), But Tokenize function returns the output as a list
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        
        # Removing Everything which is not a Alphabet
        stripped = [w.translate(table) for w in tokens]
        
         # Removing The Stop words
        words = [word for word in stripped if word.isalpha()]
#         stop_words = set(stopwords.words("english"))
#         stop_words.discard("not")    # Discarding Not from stopwords (can be important or  sentiment analysis)
#         words = [w for w in words if not w in stop_words]


        words = ' '.join(words)     # Joining the Token
        all_reviews.append(words)     # Adding into Large List.
    return all_reviews


<>:19: DeprecationWarning: invalid escape sequence \(
<>:19: DeprecationWarning: invalid escape sequence \(
<>:19: DeprecationWarning: invalid escape sequence \(
<ipython-input-309-4b35a1c3c303>:19: DeprecationWarning: invalid escape sequence \(
  pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')


In [ ]:
#Train_data["Headline"] = clean_text(Train_data["Headline"])

In [ ]:
Train_data["Headline"].iloc[0]

'AB de Villiers and wife Danielle welcomed their new-born daughter Yente to their family'

In [ ]:
def process (df, var):   # text cleaning
    
    for i in var:
        df[i] = clean_text(df[i])
    return df

In [ ]:
Train_data1 = process(Train_data, ["Headline", "articleBody"])

In [ ]:
Train_data1.head()

,Headline,Stance,articleBody
0,ab de villiers and wife danielle welcomed thei...,Agree,south africa and royal challengers bangalore b...
1,ab de villiers and wife danielle welcomed thei...,Agree,by ani pretoria former south african cricketer...
2,ab de villiers and wife danielle welcomed thei...,Agree,pretoria former south african cricketer ab de ...
3,ab de villiers and wife danielle welcomed thei...,Agree,former south african cricketer ab de villiers ...
4,ab de villiers and wife danielle welcomed thei...,Agree,former south african skipper ab de villiers an...


In [ ]:
Train_data1 = Train_data1.dropna()

In [ ]:
pwd

'C:\\Users\\Shreyansh\\Capstone_files'

In [ ]:
#Train_data1.to_csv("clean.csv")

In [ ]:
Train_data1

,Headline,Stance,articleBody
0,ab de villiers and wife danielle welcomed thei...,Agree,south africa and royal challengers bangalore b...
1,ab de villiers and wife danielle welcomed thei...,Agree,by ani pretoria former south african cricketer...
2,ab de villiers and wife danielle welcomed thei...,Agree,pretoria former south african cricketer ab de ...
3,ab de villiers and wife danielle welcomed thei...,Agree,former south african cricketer ab de villiers ...
4,ab de villiers and wife danielle welcomed thei...,Agree,former south african skipper ab de villiers an...
...,...,...,...
207,death of former lok sabha speaker sumitra mahajan,Unrelated,new delhi appreciating cooperation extended by...
208,death of former lok sabha speaker sumitra mahajan,Unrelated,former lok sabha speaker p a sangma who had a ...
209,death of former lok sabha speaker sumitra mahajan,Unrelated,outgoing lok sabha speaker and bharatiya janat...
210,death of former lok sabha speaker sumitra mahajan,Unrelated,parliament was adjourned on the first day of a...


In [ ]:
Train_data1[Train_data1.Stance ==  "unreleted"] =  "unrelated"

In [ ]:
Train_data1["Stance"].unique()

array(['Agree', 'Disagree', 'Unrelated', 'Discuss'], dtype=object)

In [ ]:
Train_data1.isnull().sum()

Headline       0
Stance         0
articleBody    0
dtype: int64

## <font color = blue> Encoding Stances

In [ ]:
Train_data1['stance_cat'] = Train_data1['Stance']

In [ ]:
Train_data1.replace({'stance_cat':{'Agree':0, 'Disagree':1 ,'Discuss':2,'Unrelated':3}}, inplace=True)

In [ ]:
type(Train_data1['Stance'])

pandas.core.series.Series

In [ ]:
Train_data1.head()

,Headline,Stance,articleBody,stance_cat
0,ab de villiers and wife danielle welcomed thei...,Agree,south africa and royal challengers bangalore b...,0
1,ab de villiers and wife danielle welcomed thei...,Agree,by ani pretoria former south african cricketer...,0
2,ab de villiers and wife danielle welcomed thei...,Agree,pretoria former south african cricketer ab de ...,0
3,ab de villiers and wife danielle welcomed thei...,Agree,former south african cricketer ab de villiers ...,0
4,ab de villiers and wife danielle welcomed thei...,Agree,former south african skipper ab de villiers an...,0


## <font color = blue>  Removing Stop Words

In [ ]:
stopwords_english = set(stopwords.words('english'))
Train_data1['Headline'] = Train_data1.Headline.apply(lambda x:str(x))
Train_data1.loc[:,'Headline'] = Train_data1['Headline'].apply(lambda x : str.lower(x))
Train_data1.loc[:,'Headline'] = Train_data1['Headline'].apply(lambda x:' '.join(re.findall('[\w]+',x)))
Train_data1.loc[:,'articleBody'] = Train_data1['articleBody'].apply(lambda x : str.lower(x))
Train_data1.loc[:,'articleBody'] = Train_data1['articleBody'].apply(lambda x:' '.join(re.findall('[\w]+',x)))


def remove_stopwords(s):
    return ' '.join(word for word in s.split() if word not in stopwords_english)

Train_data1['Headline'] = Train_data1['Headline'].apply(lambda x:remove_stopwords(x))
Train_data1['articleBody'] = Train_data1['articleBody'].apply(lambda x:str(x))
Train_data1['articleBody'] = Train_data1['articleBody'].apply(lambda x:remove_stopwords(x))

<>:4: DeprecationWarning: invalid escape sequence \w
<>:6: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \w
<>:6: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \w
<>:6: DeprecationWarning: invalid escape sequence \w
<ipython-input-326-ac370cde6c13>:4: DeprecationWarning: invalid escape sequence \w
  Train_data1.loc[:,'Headline'] = Train_data1['Headline'].apply(lambda x:' '.join(re.findall('[\w]+',x)))
<ipython-input-326-ac370cde6c13>:6: DeprecationWarning: invalid escape sequence \w
  Train_data1.loc[:,'articleBody'] = Train_data1['articleBody'].apply(lambda x:' '.join(re.findall('[\w]+',x)))


In [ ]:
Train_data1.head()

,Headline,Stance,articleBody,stance_cat
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0


In [ ]:
Train_data1['Stance'].unique()

array(['Agree', 'Disagree', 'Unrelated', 'Discuss'], dtype=object)

___

# <font color = red> Base Line Model

## <font color = blue> Creating 2 Class Data set.
- <font color = black> <b> Dividing Data set into 2 Class Unrealted and Related(Agree, Disagree and Discuss).
- Model will first check for similarilty between the user Input Claim with the Scrapped Articles and see whether they are realted or Unrealted.
- If the we are not able to find majority of articles realted to the Claim, User will prompt with the message that claim is unrelated and maximum no. of news agencies does not have anything related to the Topic.

In [ ]:
# Creating A column name Stane_Base to Divide Data set into 2 Class (Related Aand Unrelated)

#Train_data1['stance_base'] = Train_data1.loc[Train_data1.loc[:,'Stance']=='unrelated','Stance']
Train_data1['stance_base'] = Train_data1['Stance']


In [ ]:
Train_data1.replace({'stance_base':{'Agree':'related', 'Disagree':'related' ,'Discuss':'related'}}, inplace=True)
print(Train_data1['stance_base'].value_counts())

related      131
Unrelated     81
Name: stance_base, dtype: int64


In [ ]:
Train_data1.head()

,Headline,Stance,articleBody,stance_cat,stance_base
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0,related
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0,related
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0,related
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0,related
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0,related


## <font color = blue> Calculating Jacard Similarity

In [ ]:
# Creating A Function to calculate the Jaccard Similarity

def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [ ]:
def add_jaccard_similarity(data):
    count=0
    for i in range(data.shape[0]):
        jaccard_lis=[];eps=0.001
        sentence = data.loc[i,'articleBody'].split('.') #per sentence scorer
        for j in range(len(sentence)):
            jaccard_lis.append(jaccard_similarity(data.loc[i,'Headline'].split(' '),sentence[j].split(' ')))
        max_jaccard_similarity = max(jaccard_lis)
        avg_jaccard_similarity = sum(jaccard_lis)/len(jaccard_lis)
        min_jaccard_similarity = min(jaccard_lis)
        data.loc[i,'jaccard_similarity'] = (max_jaccard_similarity+min_jaccard_similarity)/(max_jaccard_similarity-min_jaccard_similarity+eps)
        if i%1000==0:
            count+=1
            print("Processed {0} Headlines".format(count*1000))


In [ ]:
add_jaccard_similarity(Train_data1)

Processed 1000 Headlines


In [ ]:
Train_data1.head()

,Headline,Stance,articleBody,stance_cat,stance_base,jaccard_similarity
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0,related,168.224299
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0,related,211.764706
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0,related,211.764706
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0,related,216.867470
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0,related,137.404580


In [ ]:
Train_data1[Train_data1["stance_base"]=="unrelated"]["jaccard_similarity"].value_counts()

Series([], Name: jaccard_similarity, dtype: int64)

In [ ]:
len(Train_data1[Train_data1["stance_base"]=="unrelated"]["jaccard_similarity"])

0

In [ ]:
#Train_data1[Train_data1["stance_base"]=="unrelated"].describe()

In [ ]:
sentence = Train_data1.loc[1,'articleBody'].split('.')

In [ ]:
sentence[0]

'ani pretoria former south african cricketer ab de villiers wife danielle de villiers welcomed baby girl righthanded batsman announced thursday de villiers mainstay royal challengers bangalore rcb batting line recently concluded ipl shared picture instagram revealed welcomed daughter november heartwarming picture shared de villers seen along wife daughter duo share smile newest addition de villiers family named yente de villiers welcomed beautiful baby girl world yente de villiers perfect addition family blessing grateful beyond measure genade onbeskryklik groot dankie de villiers captioned post edition ipl rcb bowed tournament losing sunrisers hyderabad srh eliminator righthanded batsman always lone man standing virat kohliled side showpiece event de villiers smashed runs strike rate rcb also registered five halfcenturies ipl'

In [ ]:
jaccard_lis=[]
for j in range(len(sentence)):
    jaccard_lis.append(jaccard_similarity(Train_data1.loc[2,'Headline'].split(' '),sentence[j].split(' ')))

In [ ]:
jaccard_lis

[0.10588235294117647]

In [ ]:
for j in range(len(sentence)):
    print(sentence[j].split(' '))

['ani', 'pretoria', 'former', 'south', 'african', 'cricketer', 'ab', 'de', 'villiers', 'wife', 'danielle', 'de', 'villiers', 'welcomed', 'baby', 'girl', 'righthanded', 'batsman', 'announced', 'thursday', 'de', 'villiers', 'mainstay', 'royal', 'challengers', 'bangalore', 'rcb', 'batting', 'line', 'recently', 'concluded', 'ipl', 'shared', 'picture', 'instagram', 'revealed', 'welcomed', 'daughter', 'november', 'heartwarming', 'picture', 'shared', 'de', 'villers', 'seen', 'along', 'wife', 'daughter', 'duo', 'share', 'smile', 'newest', 'addition', 'de', 'villiers', 'family', 'named', 'yente', 'de', 'villiers', 'welcomed', 'beautiful', 'baby', 'girl', 'world', 'yente', 'de', 'villiers', 'perfect', 'addition', 'family', 'blessing', 'grateful', 'beyond', 'measure', 'genade', 'onbeskryklik', 'groot', 'dankie', 'de', 'villiers', 'captioned', 'post', 'edition', 'ipl', 'rcb', 'bowed', 'tournament', 'losing', 'sunrisers', 'hyderabad', 'srh', 'eliminator', 'righthanded', 'batsman', 'always', 'lone',

In [ ]:
Train_data1.loc[1,'Headline'].split(' ')

['ab',
 'de',
 'villiers',
 'wife',
 'danielle',
 'welcomed',
 'newborn',
 'daughter',
 'yente',
 'family']

In [ ]:
sentence[0]

'ani pretoria former south african cricketer ab de villiers wife danielle de villiers welcomed baby girl righthanded batsman announced thursday de villiers mainstay royal challengers bangalore rcb batting line recently concluded ipl shared picture instagram revealed welcomed daughter november heartwarming picture shared de villers seen along wife daughter duo share smile newest addition de villiers family named yente de villiers welcomed beautiful baby girl world yente de villiers perfect addition family blessing grateful beyond measure genade onbeskryklik groot dankie de villiers captioned post edition ipl rcb bowed tournament losing sunrisers hyderabad srh eliminator righthanded batsman always lone man standing virat kohliled side showpiece event de villiers smashed runs strike rate rcb also registered five halfcenturies ipl'

In [ ]:
jaccard_similarity(Train_data1.loc[1,'Headline'].split(' '), sentence[0].split(' '))

0.10588235294117647

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Shreyansh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import nltk
lines = 'lines is some string of words'
tokenized = nltk.word_tokenize(lines)
nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if(pos[:2] == 'NN')]
print (nouns)

['lines', 'string', 'words']


In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Shreyansh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.pos_tag(tokenized)

[('lines', 'NNS'),
 ('is', 'VBZ'),
 ('some', 'DT'),
 ('string', 'NN'),
 ('of', 'IN'),
 ('words', 'NNS')]

In [ ]:
for (word, pos) in nltk.pos_tag(tokenized):
    if(pos[:2] == 'NN'):
        print(word)

lines
string
words


In [ ]:
l1  = Train_data1["Headline"]
tokenized1 = nltk.word_tokenize(l1[0])
tokenized1
nouns1 = [word for (word, pos) in nltk.pos_tag(tokenized1) if(pos[:2] == 'NN')]
print(nouns1)

['ab', 'villiers', 'wife', 'danielle', 'daughter', 'yente', 'family']


In [ ]:
headline_nouns = []
for l in l1:
    tokn1 = nltk.word_tokenize(l)
    nouns1 = [word for (word, pos) in nltk.pos_tag(tokn1) if(pos[:2] == 'NN')]
    headline_nouns.append(nouns1)
    

In [ ]:
headline_nouns

[['ab', 'villiers', 'wife', 'danielle', 'daughter', 'yente', 'family'],
 ['ab', 'villiers', 'wife', 'danielle', 'daughter', 'yente', 'family'],
 ['ab', 'villiers', 'wife', 'danielle', 'daughter', 'yente', 'family'],
 ['ab', 'villiers', 'wife', 'danielle', 'daughter', 'yente', 'family'],
 ['ab', 'villiers', 'wife', 'danielle', 'daughter', 'yente', 'family'],
 ['ab', 'villiers', 'wife', 'danielle', 'daughter', 'yente', 'family'],
 ['ab', 'villiers', 'wife', 'danielle', 'daughter', 'yente', 'family'],
 ['covidshield', 'covaccine'],
 ['covidshield', 'covaccine'],
 ['covidshield', 'covaccine'],
 ['covidshield', 'covaccine'],
 ['covidshield', 'covaccine'],
 ['covidshield', 'covaccine'],
 ['covidshield', 'covaccine'],
 ['covidshield', 'covaccine'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['april'],
 ['padukone', 'alcohol', 'lockdown'],
 ['pm'

In [ ]:
#" ".join(headline_nouns1[0])

In [ ]:
headline_nouns1 = []
for i in headline_nouns:
    headline_nouns1.append(" ".join(set(i)))

In [ ]:
Train_data2  = Train_data1.copy()

In [ ]:
Train_data2["Headline_Nouns"] = headline_nouns1

In [ ]:
Train_data2.head()

,Headline,Stance,articleBody,stance_cat,stance_base,jaccard_similarity,Headline_Nouns
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0,related,168.224299,villiers family daughter yente danielle wife ab
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0,related,211.764706,villiers family daughter yente danielle wife ab
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0,related,211.764706,villiers family daughter yente danielle wife ab
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0,related,216.867470,villiers family daughter yente danielle wife ab
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0,related,137.404580,villiers family daughter yente danielle wife ab


In [ ]:
# Creating A fucntion

def noun_extraction(df, col):
    l1_nouns = []
    l1  = df[col]
    for l in l1:
        tokn1 = nltk.word_tokenize(l)
        nouns1 = [word for (word, pos) in nltk.pos_tag(tokn1) if(pos[:2] == 'NN')]
        l1_nouns.append(nouns1)

    l1_nouns1 = []
    for i in l1_nouns:
        l1_nouns1.append(" ".join(set(i)))

    df[col + "_Nouns"] = l1_nouns1
    

In [ ]:
noun_extraction(Train_data2, "articleBody")

In [ ]:
Train_data2.head()

,Headline,Stance,articleBody,stance_cat,stance_base,jaccard_similarity,Headline_Nouns,articleBody_Nouns
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0,related,168.224299,villiers family daughter yente danielle wife ab,royal son bash girl stream danielle sons addit...
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0,related,211.764706,villiers family daughter yente danielle wife ab,duo villers girl addition groot ipl villiers e...
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0,related,211.764706,villiers family daughter yente danielle wife ab,duo villers girl addition groot ipl villiers e...
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0,related,216.867470,villiers family daughter yente danielle wife ab,duo villers girl addition groot ipl villiers e...
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0,related,137.404580,villiers family daughter yente danielle wife ab,news pandemic premier fans son bash girl strea...


In [ ]:
# Calculating Jaccrd Similarity for Two Columns With Only Nouns

def add_jaccard_similarity_noun(data):
    count=0
    for i in range(data.shape[0]):
        jaccard_lis=[];eps=0.001
        sentence = data.loc[i,'articleBody_Nouns'].split('.') #per sentence scorer
        for j in range(len(sentence)):
            jaccard_lis.append(jaccard_similarity(data.loc[i,'Headline_Nouns'].split(' '),sentence[j].split(' ')))
        max_jaccard_similarity = max(jaccard_lis)
        avg_jaccard_similarity = sum(jaccard_lis)/len(jaccard_lis)
        min_jaccard_similarity = min(jaccard_lis)
        data.loc[i,'jaccard_similarity_nouns'] = (max_jaccard_similarity+min_jaccard_similarity)/(max_jaccard_similarity-min_jaccard_similarity+eps)
        if i%1000==0:
            count+=1
            print("Processed {0} Headlines_nouns".format(count*1000))


In [ ]:
add_jaccard_similarity_noun(Train_data2)

Processed 1000 Headlines_nouns


In [ ]:
Train_data2.head()

,Headline,Stance,articleBody,stance_cat,stance_base,jaccard_similarity,Headline_Nouns,articleBody_Nouns,jaccard_similarity_nouns
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0,related,168.224299,villiers family daughter yente danielle wife ab,royal son bash girl stream danielle sons addit...,285.714286
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0,related,211.764706,villiers family daughter yente danielle wife ab,duo villers girl addition groot ipl villiers e...,315.789474
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0,related,211.764706,villiers family daughter yente danielle wife ab,duo villers girl addition groot ipl villiers e...,307.692308
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0,related,216.867470,villiers family daughter yente danielle wife ab,duo villers girl addition groot ipl villiers e...,324.324324
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0,related,137.404580,villiers family daughter yente danielle wife ab,news pandemic premier fans son bash girl strea...,190.476190


In [ ]:
Train_data2[Train_data2["stance_base"]=="unrelated"].describe()

,stance_cat,jaccard_similarity,jaccard_similarity_nouns
count,0.0,0.0,0.0
mean,NaN,NaN,NaN
std,NaN,NaN,NaN
min,NaN,NaN,NaN
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,NaN,NaN,NaN


In [ ]:
Train_data2.head()

,Headline,Stance,articleBody,stance_cat,stance_base,jaccard_similarity,Headline_Nouns,articleBody_Nouns,jaccard_similarity_nouns
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0,related,168.224299,villiers family daughter yente danielle wife ab,royal son bash girl stream danielle sons addit...,285.714286
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0,related,211.764706,villiers family daughter yente danielle wife ab,duo villers girl addition groot ipl villiers e...,315.789474
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0,related,211.764706,villiers family daughter yente danielle wife ab,duo villers girl addition groot ipl villiers e...,307.692308
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0,related,216.867470,villiers family daughter yente danielle wife ab,duo villers girl addition groot ipl villiers e...,324.324324
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0,related,137.404580,villiers family daughter yente danielle wife ab,news pandemic premier fans son bash girl strea...,190.476190


In [ ]:
Train_data2.columns

Index(['Headline', 'Stance', 'articleBody', 'stance_cat', 'stance_base',
       'jaccard_similarity', 'Headline_Nouns', 'articleBody_Nouns',
       'jaccard_similarity_nouns'],
      dtype='object')

In [ ]:
#Train_data2.to_csv("real_test_base1.csv")

___

# <font color = red> Predicting 2 Classes

## <font color = blue>  Jaccard Similarity

In [ ]:
Train_data2['stance_base'].value_counts()

related      131
Unrelated     81
Name: stance_base, dtype: int64

In [ ]:
x = Train_data2['jaccard_similarity']  # jaacard similarity as predictor
y = Train_data2['stance_base']
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.1,random_state= 101 )


In [ ]:
ytrain

42     Unrelated
112      related
121      related
66       related
183      related
         ...    
63     Unrelated
70     Unrelated
81       related
11       related
95       related
Name: stance_base, Length: 190, dtype: object

In [ ]:
print('X Training shape',xtrain.shape)
print('Y Training shape',ytrain.shape)
xtrain = xtrain.values.reshape(-1,1)
xtest = xtest.values.reshape(-1,1)

X Training shape (190,)
Y Training shape (190,)


In [ ]:
rg = RandomForestClassifier(n_estimators=100,n_jobs=-1)

In [ ]:
rg.fit(xtrain,ytrain)
ypred = rg.predict(xtest)              
print('Accuracy score on two class agree and disagree ',accuracy_score(ypred,ytest))  # test accuracy

Accuracy score on two class agree and disagree  0.7272727272727273


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(ypred,ytest))   # evaluation
print(confusion_matrix(ypred,ytest))

              precision    recall  f1-score   support

   Unrelated       0.56      0.71      0.63         7
     related       0.85      0.73      0.79        15

    accuracy                           0.73        22
   macro avg       0.70      0.72      0.71        22
weighted avg       0.75      0.73      0.73        22

[[ 5  2]
 [ 4 11]]


## <font color = blue>  Jaccard Similarity Noun

In [ ]:
Train_data2['stance_base'].value_counts()

related      131
Unrelated     81
Name: stance_base, dtype: int64

In [ ]:
x = Train_data2.iloc[:,-1]  # jaccard similarity noun as predictor
y = Train_data2['stance_base']
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.1, random_state = 101)
rg = RandomForestClassifier(n_estimators=100,n_jobs=-1)

In [ ]:
ytrain

42     Unrelated
112      related
121      related
66       related
183      related
         ...    
63     Unrelated
70     Unrelated
81       related
11       related
95       related
Name: stance_base, Length: 190, dtype: object

In [ ]:
print('X Training shape',xtrain.shape)
print('Y Training shape',ytrain.shape)
xtrain = xtrain.values.reshape(-1,1)
xtest = xtest.values.reshape(-1,1)

X Training shape (190,)
Y Training shape (190,)


In [ ]:
rg.fit(xtrain,ytrain)
ypred_noun = rg.predict(xtest)
print('Accuracy score on two class agree and disagree ',accuracy_score(ypred_noun,ytest))

Accuracy score on two class agree and disagree  0.7727272727272727


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(ypred_noun,ytest))
print(confusion_matrix(ypred_noun,ytest))

              precision    recall  f1-score   support

   Unrelated       0.56      0.83      0.67         6
     related       0.92      0.75      0.83        16

    accuracy                           0.77        22
   macro avg       0.74      0.79      0.75        22
weighted avg       0.82      0.77      0.78        22

[[ 5  1]
 [ 4 12]]


## <font color =  blue> Combined

In [ ]:
Train_data2['stance_base'].value_counts()

related      131
Unrelated     81
Name: stance_base, dtype: int64

In [ ]:
Train_data2["jaccard_similarity"].shape

(212,)

In [ ]:
Train_data2["jaccard_similarity_nouns"].shape

(212,)

In [ ]:
df = Train_data2[["jaccard_similarity", "jaccard_similarity_nouns", "stance_base"]]
df

,jaccard_similarity,jaccard_similarity_nouns,stance_base
0,168.224299,285.714286,related
1,211.764706,315.789474,related
2,211.764706,307.692308,related
3,216.867470,324.324324,related
4,137.404580,190.476190,related
...,...,...,...
207,194.444444,157.894737,Unrelated
208,53.231939,75.757576,Unrelated
209,86.956522,136.986301,Unrelated
210,48.387097,57.553957,Unrelated


In [ ]:
jaccard_similarity_nouns

In [ ]:
len(df)

212

In [ ]:
df["stance_base"].shape

(212,)

In [ ]:
x = df.iloc[:,:2]
y = df['stance_base']
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.1, random_state = 101)
rg = RandomForestClassifier(n_estimators=100,n_jobs=-1)

In [ ]:
xtrain.shape

(190, 2)

In [ ]:
ytrain

42     Unrelated
112      related
121      related
66       related
183      related
         ...    
63     Unrelated
70     Unrelated
81       related
11       related
95       related
Name: stance_base, Length: 190, dtype: object

In [ ]:
xtest.shape

(22, 2)

In [ ]:
print('X Training shape',xtrain.shape)
print('Y Training shape',ytrain.shape)
xtrain = xtrain.values.reshape(-1,1)
xtest = xtest.values.reshape(-1,1)

X Training shape (190, 2)
Y Training shape (190,)


In [ ]:
rg.fit(xtrain,ytrain)
ypred_combined = rg.predict(xtest)
print('Accuracy score on two class agree and disagree ',accuracy_score(ypred_combined,ytest))

ValueError: Number of labels=190 does not match number of samples=380

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(ypred,ytest))
print(confusion_matrix(ypred,ytest))

              precision    recall  f1-score   support

   Unrelated       0.56      0.71      0.63         7
     related       0.85      0.73      0.79        15

    accuracy                           0.73        22
   macro avg       0.70      0.72      0.71        22
weighted avg       0.75      0.73      0.73        22

[[ 5  2]
 [ 4 11]]


In [ ]:
Train_data2  =  pd.read_csv(r"C:\Users\Shreyansh\Capstone_files\final_train_data.csv")

In [ ]:
Train_data2.head()

,Unnamed: 0,Headline,Body ID,articleBody,jaccard_similarity,Headline_Nouns,articleBody_Nouns,jaccard_similarity_nouns,Headline_POS,articleBody_POS,Stance,glove_similarity,kl_divergence,ngram_overlap,semantic_similarity,stance_base
0,0,police find mass graves least bodies near mexi...,712,danny boyle directing untitled film seth rogen...,0.000000,town police mass graves bodies students clash,jobs gordon talks mark computers assistant off...,0.000000,town police mexico least find mass disappeared...,actresses mark assistant offer walter film cre...,unrelated,0.656144,1.915310,3.361780,0.167548,unrelated
1,1,hundreds palestinians flee floods gaza israel ...,158,hundreds palestinians evacuated homes sunday m...,82.840237,dams opens hundreds floods palestinians,families years lack hazard people morning open...,129.870130,dams flee hundreds gaza opens israel floods pa...,prevented save families wake rain israeli drop...,agree,0.848720,0.567896,2.895936,0.409437,related
2,2,christian bale passes role steve jobs actor re...,137,moscow resident hospitalized wounds intimate n...,0.000000,jobs bale part role passes,criminals actions adventures precision officia...,0.000000,jobs right bale christian steve part role pass...,criminals actions spend convinced adventures e...,unrelated,0.705327,1.976065,3.361780,0.233820,unrelated
3,3,hbo apple talks apple tv streaming service lau...,1034,reuters canadian soldier shot canadian war mem...,0.000000,tv apple hbo talks service april,police war amran towards teams soldiers hopkin...,0.000000,tv apple hbo talks streaming service launching...,seen police fired war amran lockdown soldier c...,unrelated,0.687689,2.065698,2.895936,0.123450,unrelated
4,4,spider burrowed tourist stomach chest,1923,fear arachnophobes story bunbury spiderman mig...,28.169014,spider stomach chest,dig mark somebody humans spider mites bodies p...,41.237113,tourist stomach chest spider burrowed,seen dig mark somebody humans spider mites bod...,disagree,0.584935,0.379117,2.244121,0.510268,related


# LDA

In [ ]:



def noun_extraction_pos(df, col):
    tags_to_keep = ['JJ', 'NN', 'NNP', 'NNPS', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    
    l1_nouns = []
    l1  = df[col]
    for l in l1:
        tokn1 = nltk.word_tokenize(l)
        nouns1 = [word for (word, pos) in nltk.pos_tag(tokn1) if(pos in tags_to_keep)]
        l1_nouns.append(nouns1)

    l1_nouns1 = []
    for i in l1_nouns:
        l1_nouns1.append(" ".join(set(i)))
    df[f"{col}_pos"] = l1_nouns1


In [ ]:
noun_extraction_pos(Train_data2, "Headline")

In [ ]:
Train_data2.head()

,Headline,Stance,articleBody,stance_cat,stance_base,jaccard_similarity,Headline_pos
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0,related,168.224299,villiers welcomed newborn family daughter yent...
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0,related,211.764706,villiers welcomed newborn family daughter yent...
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0,related,211.764706,villiers welcomed newborn family daughter yent...
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0,related,216.867470,villiers welcomed newborn family daughter yent...
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0,related,137.404580,villiers welcomed newborn family daughter yent...


In [ ]:
noun_extraction_pos(Train_data2, "articleBody")

In [ ]:
Train_data2.head()

,Headline,Stance,articleBody,stance_cat,stance_base,jaccard_similarity,Headline_pos,articleBody_pos
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0,related,168.224299,villiers welcomed newborn family daughter yent...,welcomed bangalore photo addition parents succ...
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0,related,211.764706,villiers welcomed newborn family daughter yent...,smile welcomed royal duo villers girl standing...
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0,related,211.764706,villiers welcomed newborn family daughter yent...,smile welcomed royal duo villers girl standing...
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0,related,216.867470,villiers welcomed newborn family daughter yent...,smile welcomed royal duo villers girl standing...
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0,related,137.404580,villiers welcomed newborn family daughter yent...,pulled welcomed fans postponed bangalore india...


In [ ]:
Train_data2['jaccard_similarity_nouns'] = df['jaccard_similarity_nouns']

In [ ]:
Train_data2.columns

Index(['Headline', 'Stance', 'articleBody', 'stance_cat', 'stance_base',
       'jaccard_similarity', 'Headline_pos', 'articleBody_pos',
       'jaccard_similarity_nouns'],
      dtype='object')

In [ ]:
Train_data3 = Train_data2.copy()

In [ ]:
Train_data2["Headline_Nouns"] = headline_nouns1   # consider Headline nouns

In [ ]:
Train_data2.columns

Index(['Headline', 'Stance', 'articleBody', 'stance_cat', 'stance_base',
       'jaccard_similarity', 'Headline_pos', 'articleBody_pos',
       'jaccard_similarity_nouns', 'Headline_Nouns'],
      dtype='object')

In [ ]:
def lemmatization(data,col):   # lemmatization of words
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    def lemmatize_text(text):
        return [lemmatizer.lemmatize(w , pos = 'v') for w in w_tokenizer.tokenize(text)]

    df = pd.DataFrame(data[col])
 
    data[col +'_lemmatized'] = df[col].apply(lemmatize_text)
    
    l = []
    for i in data[col +'_lemmatized']:
        l.append(" ".join(i))
    data[col +'_lemmatized'] = l

    

In [ ]:
lemmatization(Train_data2 ,"articleBody_pos" )

In [ ]:
lemmatization(Train_data2 ,"Headline_pos" )

In [ ]:
Train_data2.head()

,Headline,Stance,articleBody,stance_cat,stance_base,jaccard_similarity,Headline_pos,articleBody_pos,jaccard_similarity_nouns,Headline_Nouns,articleBody_pos_lemmatized,Headline_pos_lemmatized
0,ab de villiers wife danielle welcomed newborn ...,Agree,south africa royal challengers bangalore batsm...,0,related,168.224299,villiers welcomed newborn family daughter yent...,welcomed bangalore photo addition parents succ...,285.714286,villiers family daughter yente danielle wife ab,welcome bangalore photo addition parent succes...,villiers welcome newborn family daughter yente...
1,ab de villiers wife danielle welcomed newborn ...,Agree,ani pretoria former south african cricketer ab...,0,related,211.764706,villiers welcomed newborn family daughter yent...,smile welcomed royal duo villers girl standing...,315.789474,villiers family daughter yente danielle wife ab,smile welcome royal duo villers girl stand ban...,villiers welcome newborn family daughter yente...
2,ab de villiers wife danielle welcomed newborn ...,Agree,pretoria former south african cricketer ab de ...,0,related,211.764706,villiers welcomed newborn family daughter yent...,smile welcomed royal duo villers girl standing...,307.692308,villiers family daughter yente danielle wife ab,smile welcome royal duo villers girl stand ban...,villiers welcome newborn family daughter yente...
3,ab de villiers wife danielle welcomed newborn ...,Agree,former south african cricketer ab de villiers ...,0,related,216.867470,villiers welcomed newborn family daughter yent...,smile welcomed royal duo villers girl standing...,324.324324,villiers family daughter yente danielle wife ab,smile welcome royal duo villers girl stand ban...,villiers welcome newborn family daughter yente...
4,ab de villiers wife danielle welcomed newborn ...,Agree,former south african skipper ab de villiers wi...,0,related,137.404580,villiers welcomed newborn family daughter yent...,pulled welcomed fans postponed bangalore india...,190.476190,villiers family daughter yente danielle wife ab,pull welcome fan postpone bangalore indian int...,villiers welcome newborn family daughter yente...


In [ ]:
# DATASET_PATH = os.path.join(os.getcwd(),"C:\\Users\\Shreyansh\\Abhinav\data","")

# Train_data2 = pd.read_csv(DATASET_PATH+"Train_data2_combined.csv",encoding='ISO-8859-1')

In [ ]:
Train_data2.columns

Index(['Headline', 'Stance', 'articleBody', 'stance_cat', 'stance_base',
       'jaccard_similarity', 'Headline_pos', 'articleBody_pos',
       'jaccard_similarity_nouns', 'Headline_Nouns',
       'articleBody_pos_lemmatized', 'Headline_pos_lemmatized'],
      dtype='object')

In [ ]:
nf = Train_data2[["Headline_POS" , "articleBody_POS"]] 

In [ ]:
filtered_texts = nf["Headline_POS"]

In [ ]:
def LDA_Conversion_Matrix(column_name , no_of_topics):
    #modules to build the topic extracting models
    from gensim import corpora, models
    import gensim
    import pyLDAvis.gensim
    pyLDAvis.enable_notebook()
    import pandas as pd
    import matplotlib.pyplot as plt
    
    filtered_texts = list(column_name)
    #creating the dictionary
    dictionary = corpora.Dictionary([" ".join(filtered_texts).split()]) 
    #print('{} different terms in the corpus'.format(len(dictionary)))
    
    #creating the bag of words object
    
    
    bow_corpus = [dictionary.doc2bow(text.split()) for text in filtered_texts]
    tfidf_model = models.TfidfModel(bow_corpus) # creating the tf-idf model
    tfidf_corpus = tfidf_model[bow_corpus]
    
    total_topics = no_of_topics
    lda_model_tfidf = models.LdaModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=total_topics, 
                                  passes=1, random_state=47)
    lda_model_bow = models.LdaModel(corpus=bow_corpus, id2word=dictionary, num_topics=total_topics,
                                passes=1, random_state=47)
    
    
    data = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)
    #print(data)
    
    
    new_basket = []   
    for j in filtered_texts:
        z = j
        vec_bow = dictionary.doc2bow(z.split())
        new_basket.append(vec_bow)
        
        
    vec_lda_topics_basket = []
    for i in new_basket:
        vec_lda_topics = lda_model_bow[i]
        vec_lda_topics_basket.append(vec_lda_topics)
    vec_lda_topics_basket
    
    
    
    #df = pd.DataFrame(vec_lda_topics_basket)
    #dff = df.stack().unstack(fill_value=(0, 0))
    
    return vec_lda_topics_basket


In [ ]:
body_LDA = LDA_Conversion_Matrix( nf["articleBody_POS"] , 25)  # LDA score is calculated for (articles Parts of speech tagged ) and articles POS is formed into 25 clusters

C:\ProgramData\Anaconda3\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\ProgramData\Anaconda3\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


In [ ]:
headline_LDA = LDA_Conversion_Matrix( nf["Headline_POS"] , 25)  # LDA score is calculated for (Headline Parts of speech tagged ) and Headline POS is formed into 25 clusters

In [ ]:
len(body_LDA)

75385

In [ ]:
len(headline_LDA)

75385

In [ ]:
body_LDA[1]

[(1, 0.014937095),
 (5, 0.1084584),
 (6, 0.024180355),
 (7, 0.09436764),
 (8, 0.06832374),
 (13, 0.21895494),
 (16, 0.05352859),
 (17, 0.03672689),
 (20, 0.013868878),
 (21, 0.36266696)]

In [ ]:
headline_LDA[1]

[(22, 0.893318)]

In [ ]:
import gensim
from gensim import corpora
from gensim import models
from gensim.models import LdaModel
from gensim.models import TfidfModel
from gensim.models import CoherenceModel

In [ ]:
masket = []
for i in range(len(headline_LDA)):
    simmilarity = gensim.matutils.cossim(headline_LDA[i], body_LDA[i])
    masket.append(simmilarity)
len(masket)

75385

In [ ]:
masket

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.04774977550043799,
 0.0,
 0.0,
 0.0,
 0.0,
 0.028769211429950318,
 0.021454825665785257,
 0.0,
 0.13060725232795917,
 0.05880060755373019,
 0.0,
 0.0,
 0.0,
 0.3074746110427348,
 0.0,
 0.0,
 0.0,
 0.1445734933343478,
 0.4725068205964438,
 0.0,
 0.0,
 0.006564476197540084,
 0.31335646366249464,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.15163348233466642,
 0.0,
 0.2641518059804959,
 0.0,
 0.0,
 0.045453568964596514,
 0.3310114166387529,
 0.0912259586175136,
 0.013131234540366047,
 0.35956765163859533,
 0.0,
 0.0,
 0.05087036306988893,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.09398482580834541,
 0.06287793874294252,
 0.021326382231501743,
 0.0,
 0.15931861478477732,
 0.02183246228153673,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.958269645396889,
 0.0,
 0.0,
 0.0,
 0.01466607341002623,
 0.2314524996104179,
 0.0,
 0.0,
 0.0,
 0.027718631717102704,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.01742209873230496,
 0.0,
 0.0,
 0.8782162731032785,
 0.0,
 0.0,
 0.08555787894430515,
 

In [ ]:
Train_data2["LDA_Score"] = pd.DataFrame(masket)
Train_data2.columns

Index(['Unnamed: 0', 'Headline', 'Body ID', 'articleBody',
       'jaccard_similarity', 'Headline_Nouns', 'articleBody_Nouns',
       'jaccard_similarity_nouns', 'Headline_POS', 'articleBody_POS', 'Stance',
       'glove_similarity', 'kl_divergence', 'ngram_overlap',
       'semantic_similarity', 'stance_base', 'LDA_Score'],
      dtype='object')

In [ ]:
Train_data_new  = Train_data2[["Headline" ,"articleBody" ,"stance_base" ,"LDA_Score" , "jaccard_similarity", "jaccard_similarity_nouns"]]
Train_data_new.head(5)

,Headline,articleBody,stance_base,LDA_Score,jaccard_similarity,jaccard_similarity_nouns
0,police find mass graves least bodies near mexi...,danny boyle directing untitled film seth rogen...,unrelated,0.0,0.000000,0.000000
1,hundreds palestinians flee floods gaza israel ...,hundreds palestinians evacuated homes sunday m...,related,0.0,82.840237,129.870130
2,christian bale passes role steve jobs actor re...,moscow resident hospitalized wounds intimate n...,unrelated,0.0,0.000000,0.000000
3,hbo apple talks apple tv streaming service lau...,reuters canadian soldier shot canadian war mem...,unrelated,0.0,0.000000,0.000000
4,spider burrowed tourist stomach chest,fear arachnophobes story bunbury spiderman mig...,related,0.0,28.169014,41.237113


In [ ]:
Train_data2.columns

Index(['Unnamed: 0', 'Headline', 'Body ID', 'articleBody',
       'jaccard_similarity', 'Headline_Nouns', 'articleBody_Nouns',
       'jaccard_similarity_nouns', 'Headline_POS', 'articleBody_POS', 'Stance',
       'glove_similarity', 'kl_divergence', 'ngram_overlap',
       'semantic_similarity', 'stance_base', 'LDA_Score'],
      dtype='object')

In [ ]:
Train_data2.shape

(75385, 17)

In [ ]:
#Train_data2.to_csv("final_train_data.csv")

In [ ]:
x = Train_data2[['LDA_Score',"jaccard_similarity", "jaccard_similarity_nouns"]]
y = Train_data2['stance_base']
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.1,random_state= 101 )


In [ ]:
ytrain

42     Unrelated
112      related
121      related
66       related
183      related
         ...    
63     Unrelated
70     Unrelated
81       related
11       related
95       related
Name: stance_base, Length: 190, dtype: object

In [ ]:
print('X Training shape',xtrain.shape)
print('Y Training shape',ytrain.shape)
xtrain = xtrain.values.reshape(-1,1)
xtest = xtest.values.reshape(-1,1)

X Training shape (190, 3)
Y Training shape (190,)


In [ ]:
rg = RandomForestClassifier(n_estimators=100,n_jobs=-1)

In [ ]:
# rg.fit(xtrain,ytrain)
# ypred = rg.predict(xtest)
# print('Accuracy score on two class agree and disagree ',accuracy_score(ypred,ytest))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(ypred,ytest))
print(confusion_matrix(ypred,ytest))

              precision    recall  f1-score   support

   Unrelated       0.56      0.71      0.63         7
     related       0.85      0.73      0.79        15

    accuracy                           0.73        22
   macro avg       0.70      0.72      0.71        22
weighted avg       0.75      0.73      0.73        22

[[ 5  2]
 [ 4 11]]


In [ ]:
#filtered_texts = pd.DataFrame(filtered_texts)

In [ ]:
filtered_texts = list(filtered_texts)

In [ ]:
filtered_texts

['town police mexico least find mass disappeared graves bodies students clash',
 'dams flee hundreds gaza opens israel floods palestinians',
 'jobs right bale christian steve part role passes felt nt actor',
 'tv apple hbo talks streaming service launching april',
 'tourist stomach chest spider burrowed',
 'goes confirms news total december experience days story viral earth fake darkness',
 'accused prison marathon walk bomber talk injured boston',
 'revealed terrorist jihadi identity john known isis',
 'revealed hoax arrested identity year last real',
 'aid murdered worker isis british confirmed',
 'pundit gateway',
 'woman wife albaghdadi lebanon detained iraq says',
 'schoolgirls ceasefire nigerian government deal haram home girls missing bring claims kidnapped boko',
 'kid high make school trading stocks',
 'york arrest new marijuana lead ticket',
 'add turned vandals job paint hoax bugatti owner rude',
 'denies boko ceasefire nigeria claim haram',
 'rip plant robert contract',
 'c

### Building the models to find correlated concepts

In [ ]:
#modules to build the topic extracting models
from gensim import corpora, models
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#creating the dictionary
dictionary = corpora.Dictionary([" ".join(filtered_texts).split()]) 
print('{} different terms in the corpus'.format(len(dictionary)))
#creating the bag of words object
bow_corpus = [dictionary.doc2bow(text.split()) for text in filtered_texts]

3070 different terms in the corpus


In [ ]:
len(bow_corpus)

49972

In [ ]:
tfidf_model = models.TfidfModel(bow_corpus) # creating the tf-idf model
tfidf_corpus = tfidf_model[bow_corpus]

### Building the LDA models.

In [ ]:
total_topics = 50
lda_model_tfidf = models.LdaModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=total_topics, 
                                  passes=1, random_state=47)
lda_model_bow = models.LdaModel(corpus=bow_corpus, id2word=dictionary, num_topics=total_topics,
                                passes=1, random_state=47)

In [ ]:
data = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)
data

C:\ProgramData\Anaconda3\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\ProgramData\Anaconda3\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2      0.262363  0.120309       1        1  5.145613
39    -0.005661 -0.086137       2        1  3.953743
9     -0.067054 -0.103774       3        1  3.067791
8      0.181508  0.016181       4        1  2.979521
11    -0.226728  0.196713       5        1  2.868921
47    -0.033054 -0.194364       6        1  2.833104
21    -0.066491  0.078202       7        1  2.525541
40    -0.080519  0.041354       8        1  2.438658
26     0.266737  0.121110       9        1  2.422625
22     0.040863 -0.032451      10        1  2.376890
29    -0.063845 -0.145410      11        1  2.361221
42    -0.087540 -0.075069      12        1  2.282275
15     0.154865  0.077797      13        1  2.217219
48     0.124049 -0.005268      14        1  2.205208
1     -0.141128  0.085709      15        1  2.199844
27     0.019611 -0.042038      16        1  2.170815
35    -0.176910  0.138033      17        1  2.115478
13    -0.115501 -0.040648      18        1  2.114540
33    -0.021733 -0.060546      19        1  2.073616
10     0.125414  0.029225      20        1  2.046483
41    -0.013135 -0.032714      21        1  2.011644
32    -0.212110  0.212118      22        1  1.978853
43     0.085189  0.050708      23        1  1.977090
20    -0.091188 -0.008563      24        1  1.864183
46    -0.034084 -0.025701      25        1  1.854852
37    -0.052195 -0.086574      26        1  1.838377
4      0.103398  0.007430      27        1  1.818403
18     0.139829  0.024170      28        1  1.811287
34     0.092290  0.032686      29        1  1.787961
25    -0.110209  0.233969      30        1  1.755759
6     -0.144413  0.149557      31        1  1.749574
28    -0.105077  0.100209      32        1  1.724495
38    -0.052568 -0.050261      33        1  1.722474
36    -0.097268  0.098159      34        1  1.625480
24     0.166041  0.031947      35        1  1.625374
31    -0.059153 -0.163971      36        1  1.593277
19     0.028764 -0.020452      37        1  1.538337
44     0.075218 -0.008393      38        1  1.524670
14    -0.028033 -0.087604      39        1  1.471517
17    -0.035190  0.032808      40        1  1.457770
49     0.129659  0.113273      41        1  1.426807
23     0.081157 -0.081537      42        1  1.420439
16    -0.055775 -0.152654      43        1  1.416861
45    -0.054607 -0.223821      44        1  1.411096
0     -0.025355 -0.072992      45        1  1.375960
12     0.093095 -0.015078      46        1  1.246530
30    -0.051851 -0.071328      47        1  1.242139
3     -0.065133 -0.099203      48        1  1.200928
5      0.197104  0.060022      49        1  1.085712
7      0.006353 -0.065138      50        1  1.043043, topic_info=            Term         Freq        Total Category  logprob  loglift
1371        isis  7887.000000  7887.000000  Default  30.0000  30.0000
1422  journalist  2815.000000  2815.000000  Default  29.0000  29.0000
1198       haram  2363.000000  2363.000000  Default  28.0000  28.0000
308         boko  2363.000000  2363.000000  Default  27.0000  27.0000
1654     michael  2977.000000  2977.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
2708       texas    72.499617   560.735739  Topic50  -3.9946   2.5174
200         baby    75.833266   693.787968  Topic50  -3.9496   2.3494
411     captured    71.218468  1020.239856  Topic50  -4.0124   1.9010
2698      terror    56.453317   424.917963  Topic50  -4.2448   2.5445
266          big    57.686360   590.448401  Topic50  -4.2232   2.2372

[2449 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         3  0.981749       aaron
1        37  0.995461   abandoned
2        28  0.404270       abdel
2        35  0.592171       abdel
3        28  0.370027  abdelmajed
...     ...       ...         ...
3064      3  0.994251    zeppelin
3065     19  0.97

### Calculating the topics for documents 03, 12 and 17

In [ ]:
#basket = []
#for i in filtered_texts:
    #basket.append(i)
new_basket = []   
for j in filtered_texts:
    z = j
    vec_bow = dictionary.doc2bow(z.split())
    new_basket.append(vec_bow)
    
new_basket[17]
    
    

[(600, 1), (1956, 1), (2238, 1), (2248, 1)]

In [ ]:
y = 'didnt robert rip plant contract'

In [ ]:
y = filtered_texts[17]

In [ ]:
vec_bow_179 = dictionary.doc2bow(y.split())

In [ ]:
vec_lda_topics_basket = []
for i in new_basket:
    vec_lda_topics = lda_model_bow[i]
    vec_lda_topics_basket.append(vec_lda_topics)
vec_lda_topics_basket    

[[(27, 0.3348677), (38, 0.58512986)],
 [(16, 0.78001714), (29, 0.113311775)],
 [(7, 0.5679284), (9, 0.3520707)],
 [(14, 0.89111036)],
 [(40, 0.83666575)],
 [(19, 0.9246135)],
 [(33, 0.8911098)],
 [(6, 0.12749125), (12, 0.16246559), (32, 0.2524995), (43, 0.3425408)],
 [(20, 0.5015746), (32, 0.37842375)],
 [(4, 0.15875272), (15, 0.55964375), (19, 0.1473157)],
 [(0, 0.02),
  (1, 0.02),
  (2, 0.02),
  (3, 0.02),
  (4, 0.02),
  (5, 0.02),
  (6, 0.02),
  (7, 0.02),
  (8, 0.02),
  (9, 0.02),
  (10, 0.02),
  (11, 0.02),
  (12, 0.02),
  (13, 0.02),
  (14, 0.02),
  (15, 0.02),
  (16, 0.02),
  (17, 0.02),
  (18, 0.02),
  (19, 0.02),
  (20, 0.02),
  (21, 0.02),
  (22, 0.02),
  (23, 0.02),
  (24, 0.02),
  (25, 0.02),
  (26, 0.02),
  (27, 0.02),
  (28, 0.02),
  (29, 0.02),
  (30, 0.02),
  (31, 0.02),
  (32, 0.02),
  (33, 0.02),
  (34, 0.02),
  (35, 0.02),
  (36, 0.02),
  (37, 0.02),
  (38, 0.02),
  (39, 0.02),
  (40, 0.02),
  (41, 0.02),
  (42, 0.02),
  (43, 0.02),
  (44, 0.02),
  (45, 0.02),
  (46,

In [ ]:
vec_bow_17 = dictionary.doc2bow(filtered_texts[17].split())  # creating corpus
vec_bow_03 = dictionary.doc2bow(filtered_texts[3].split())
vec_bow_12 = dictionary.doc2bow(filtered_texts[12].split())

In [ ]:
vec_bow_17

[(600, 1), (1956, 1), (2238, 1), (2248, 1)]

### Viewing the topics of the selected documents

In [ ]:
vec_lda_topics_17 = lda_model_bow[vec_bow_17]
vec_lda_topics_03 = lda_model_bow[vec_bow_03]
vec_lda_topics_12 = lda_model_bow[vec_bow_12]
print ('document 03 topics: ', vec_lda_topics_03)
print ('document 12 topics: ', vec_lda_topics_12)
print ('document 17 topics: ', vec_lda_topics_17)

document 03 topics:  [(14, 0.89111036)]
document 12 topics:  [(39, 0.9299994)]
document 17 topics:  [(9, 0.8039964)]


In [ ]:
df1 = pd.DataFrame(vec_lda_topics_03, columns=['topic', 'contrib'])
df1['doc'] = 'doc_03'
df2 = pd.DataFrame(vec_lda_topics_12, columns=['topic', 'contrib'])
df2['doc'] = 'doc_12'
df3 = pd.DataFrame(vec_lda_topics_17, columns=['topic', 'contrib'])
df3['doc'] = 'doc_17'

In [ ]:
dff = dff.stack().unstack(fill_value=(0, 0))

In [ ]:
#vec_lda_topics_basket

In [ ]:
dff = pd.DataFrame(vec_lda_topics_basket)
dff.head(50)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,"(27, 0.3348677)","(38, 0.58512986)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"(16, 0.78001714)","(29, 0.113311775)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"(7, 0.5679284)","(9, 0.3520707)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"(14, 0.89111036)",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"(40, 0.83666575)",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"(19, 0.9246135)",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"(33, 0.8911098)",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,"(6, 0.12749125)","(12, 0.16246559)","(32, 0.2524995)","(43, 0.3425408)",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,"(20, 0.5015746)","(32, 0.37842375)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,"(4, 0.15875272)","(15, 0.55964375)","(19, 0.1473157)",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### Calculating the simmilarity between the documents
Documents 3 and 17 are closer to one another than 3 is to 12 or 12 is to 17.

In [ ]:
dff.iloc[5]

0     (19, 0.9246135)
1              (0, 0)
2              (0, 0)
3              (0, 0)
4              (0, 0)
5              (0, 0)
6              (0, 0)
7              (0, 0)
8              (0, 0)
9              (0, 0)
10             (0, 0)
11             (0, 0)
12             (0, 0)
13             (0, 0)
14             (0, 0)
15             (0, 0)
16             (0, 0)
17             (0, 0)
18             (0, 0)
19             (0, 0)
20             (0, 0)
21             (0, 0)
22             (0, 0)
23             (0, 0)
24             (0, 0)
25             (0, 0)
26             (0, 0)
27             (0, 0)
28             (0, 0)
29             (0, 0)
30             (0, 0)
31             (0, 0)
32             (0, 0)
33             (0, 0)
34             (0, 0)
35             (0, 0)
36             (0, 0)
37             (0, 0)
38             (0, 0)
39             (0, 0)
40             (0, 0)
41             (0, 0)
42             (0, 0)
43             (0, 0)
44             (0, 0)
45        

In [ ]:
list(dff.iloc[3])

[(14, 0.89111036),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0)]

In [ ]:
simmilarity_03_12 = gensim.matutils.cossim(vec_lda_topics_basket[3], vec_lda_topics_basket[3])
#simmilarity_03_17 = gensim.matutils.cossim(vec_lda_topics_03, vec_lda_topics_17)
#simmilarity_12_17 = gensim.matutils.cossim(vec_lda_topics_12, vec_lda_topics_17)

In [ ]:
print('similarity between docs 03 and 12', simmilarity_03_12)
#print('similarity between docs 03 and 17', simmilarity_03_17)
#print('similarity between docs 12 and 17', simmilarity_12_17)

similarity between docs 03 and 12 1.0


In [ ]:
null_frame = dff.copy()

In [ ]:
null_frame2 = pd.DataFrame(columns=dff.columns, index=dff.index)

In [ ]:
vec_lda_topics_basket

[[(27, 0.3348677), (38, 0.58512986)],
 [(16, 0.78001714), (29, 0.113311775)],
 [(7, 0.5679284), (9, 0.3520707)],
 [(14, 0.89111036)],
 [(40, 0.83666575)],
 [(19, 0.9246135)],
 [(33, 0.8911098)],
 [(6, 0.12749125), (12, 0.16246559), (32, 0.2524995), (43, 0.3425408)],
 [(20, 0.5015746), (32, 0.37842375)],
 [(4, 0.15875272), (15, 0.55964375), (19, 0.1473157)],
 [(0, 0.02),
  (1, 0.02),
  (2, 0.02),
  (3, 0.02),
  (4, 0.02),
  (5, 0.02),
  (6, 0.02),
  (7, 0.02),
  (8, 0.02),
  (9, 0.02),
  (10, 0.02),
  (11, 0.02),
  (12, 0.02),
  (13, 0.02),
  (14, 0.02),
  (15, 0.02),
  (16, 0.02),
  (17, 0.02),
  (18, 0.02),
  (19, 0.02),
  (20, 0.02),
  (21, 0.02),
  (22, 0.02),
  (23, 0.02),
  (24, 0.02),
  (25, 0.02),
  (26, 0.02),
  (27, 0.02),
  (28, 0.02),
  (29, 0.02),
  (30, 0.02),
  (31, 0.02),
  (32, 0.02),
  (33, 0.02),
  (34, 0.02),
  (35, 0.02),
  (36, 0.02),
  (37, 0.02),
  (38, 0.02),
  (39, 0.02),
  (40, 0.02),
  (41, 0.02),
  (42, 0.02),
  (43, 0.02),
  (44, 0.02),
  (45, 0.02),
  (46,

In [ ]:
vec_lda_topics_basket[0]

[(27, 0.3348677), (38, 0.58512986)]

In [ ]:
basket_for_cosine = []
for j in range(len(vec_lda_topics_basket)):
    for i in vec_lda_topics_basket[j]:
        index = vec_lda_topics_basket.index(vec_lda_topics_basket[j])
        add = [index] + list(i)
        basket_for_cosine.append(add)
    
#basket

In [ ]:
cosine_matrix = pd.DataFrame(basket_for_cosine, columns =['row_index', 'column_index', 'Score']) 
  

In [ ]:
cosine_matrix.to_csv('LDA_frame1.csv') 

In [ ]:
cosine_matrix.head()

,row_index,column_index,Score
0,0,27,0.3348676860332489
1,0,38,0.5851298570632935
2,1,16,0.7800171375274658
3,1,29,0.1133117750287056
4,2,7,0.5679283738136292


In [ ]:
cosine_matrix['row_index']= cosine_matrix['row_index'].map(str)
cosine_matrix['column_index']= cosine_matrix['column_index'].map(str)
cosine_matrix['Score']= cosine_matrix['Score'].map(str)

In [ ]:
a = cosine_matrix.copy()
b = a.drop_duplicates()
b.shape

In [ ]:
b.head()

In [ ]:
len(basket)

In [ ]:
x = [1,2]
y = [3,4]
x + y

In [ ]:
null_frame2

In [ ]:
df.to_csv(index=False)

In [ ]:
nf.to_csv(r"C:\Users\Shreyansh\Abhinav\feature_selection_matrix.csv", encoding='ISO-8859-1')